# Assignment 4
## Description

In this assignment you must read in a file of metropolitan regions and associated sports teams from assets/wikipedia_data.html and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in assets/nfl.csv), MLB (baseball, in assets/mlb.csv), NBA (basketball, in assets/nba.csv or NHL (hockey, in assets/nhl.csv). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: what is the win/loss ratio's correlation with the population of the city it is in? Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with pearsonr, so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%*4=80%) of the grade for this assignment. You should only use data from year 2018 for your analysis -- this is important!

In [1]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [3]:
import pandas as pd

cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [6]:
# Cleaning cities dataframe
import re
for x in cities:
    cities[x] = cities[x].str.replace("\[.+\]|—", "", regex=True)
    cities.rename(columns={x : re.sub("[\(\[].+[\)\]]", "", x).strip()}, inplace=True)
cities.head()

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets,Yankees Mets,Knicks Nets,Rangers Islanders Devils
1,Los Angeles,13310447,Rams Chargers,Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders,Giants Athletics,Warriors,Sharks
3,Chicago,9512999,Bears,Cubs White Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


## Question 1

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NHL using 2018 data.


In [9]:
nhl_df = pd.read_csv("assets/nhl.csv")
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


In [11]:
def nhl_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re
    
    nhl_df = pd.read_csv("assets/nhl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    for x in cities:
        cities[x] = cities[x].str.replace("\[.+\]|—", "", regex=True)
        cities.rename(columns={x : re.sub("[\(\[].+[\)\]]", "", x).strip()}, inplace=True)
    nhl_df["team"] = nhl_df["team"].str.replace("[*]", "", regex=True)
    
    team_names = {}
    i = 0
    for names in cities["NHL"]:
        for name in names.split():
            if len(name) < 1: continue
            team_names[name] = cities["Metropolitan area"].iloc[i]
        i = i + 1
        
    nhl_df["area"] = ""
    for x in team_names:
        nhl_df.loc[nhl_df["team"].str.contains(r"\b{}\b".format(x), regex=True), "area"] = team_names[x]
    
    nhl_merge = pd.merge(nhl_df[nhl_df["year"]==2018][["area", "team", "W", "L", "year"]],
                         cities[["Metropolitan area", "Population"]],
                         left_on="area", right_on="Metropolitan area")
    nhl_merge = nhl_merge.astype({"W": int, "L": int, "Population": int})
    nhl_merge["W/L Ratio"] = nhl_merge["W"] / (nhl_merge["W"] + nhl_merge["L"])
    
    grouped = nhl_merge.groupby(by="area").agg({"W/L Ratio": np.mean, "Population": np.mean})
    
    #raise NotImplementedError()
    
    population_by_region = grouped["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = grouped["W/L Ratio"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

nhl_correlation()

0.012486162921209881

## Question 2

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NBA using 2018 data.


In [13]:
nba_df=pd.read_csv("assets/nba.csv")
nba_df.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA


In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def nba_correlation():
    for x in cities:
        cities[x] = cities[x].str.replace("\[.+\]|—", "", regex=True)
        cities.rename(columns={x : re.sub("[\(\[].+[\)\]]", "", x).strip()}, inplace=True)

    nba_df["team"] = nba_df["team"].str.replace("[*\(].+", "", regex=True).str.strip()
    
    team_names = {}
    i = 0
    for names in cities["NBA"]:
        for name in names.split():
            if len(name) < 1: continue
            team_names[name] = cities["Metropolitan area"].iloc[i]
        i = i + 1
    #print("Dict \n{}".format(team_names))

    nba_df["area"] = ""
    for x in team_names:
        nba_df.loc[nba_df["team"].str.contains(r"\b{}\b".format(x), regex=True), "area"] = team_names[x]

    nba_merge = pd.merge(nba_df[nba_df["year"]==2018][["area", "team", "W", "L", "W/L%", "year"]],
                        cities[["Metropolitan area", "Population"]],
                        left_on="area", right_on="Metropolitan area")
    nba_merge = nba_merge.astype({"W": int, "L": int, "W/L%": float, "Population": int})
    
    grouped = nba_merge.groupby(by="area").agg({"W/L%": np.mean, "Population": np.mean})
    #print(grouped)
    
    population_by_region = grouped["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = grouped["W/L%"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

nba_correlation()

-0.1763635064218294

## Question 5
In this question I would like you to explore the hypothesis that given that an area has two sports teams in different sports, those teams will perform the same within their respective sports. How I would like to see this explored is with a series of paired t-tests (so use ttest_rel) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.


In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def sports_team_performance():
    for x in cities:
        cities[x] = cities[x].str.replace("\[.+\]|—", "", regex=True)
        cities.rename(columns={x : re.sub("[\(\[].+[\)\]]", "", x).strip()}, inplace=True)

    nhl_df["team"] = nhl_df["team"].str.replace("[*]", "", regex=True)
    nba_df["team"] = nba_df["team"].str.replace("[*\(].+", "", regex=True).str.strip()
    mlb_df["team"] = mlb_df["team"].str.strip()
    nfl_df["team"] = nfl_df["team"].str.replace("[*+]", "", regex=True).str.strip()
    
    Big4 = {"NHL": nhl_df, "NBA": nba_df, "MLB": mlb_df, "NFL": nfl_df}
    team_names = {}
    team_names_list = []
    for sport in Big4: #itera solo en las keys()
        i = 0
        for names in cities[sport]:
            if i > len(cities[sport]): break
            for name in names.split():
                if len(name) <= 1: continue
                city_region = cities["Metropolitan area"].iloc[i]
                #team_names[name] = cities["Metropolitan area"].iloc[i] #dict_keys no son unique/Ej Rangers en NHl y MBL
                team_names_list.append((name, city_region))
            i += 1
        Big4[sport]["area"] = "" ## Big4[sport]["area"] pasa a ser xxx_df["area"]
        for x in team_names_list:
            if x[0] == "Sox": continue
            regex_search = r"\b{}\b".format(x[0])
            Big4[sport].loc[Big4[sport]["team"].str.contains(regex_search, regex=True), "area"] = x[1]
            
    
    ## Comprobaciones
    #print(nhl_df[["team", "area"]][nhl_df["area"]=="Las Vegas"])
    #print(nhl_df[["team", "area"]][nhl_df["team"].str.contains("Rangers")])
    #print(mlb_df[["team", "area"]][mlb_df["team"].str.contains("Rangers")])
    #print(nfl_df[["team", "area"]][nfl_df["area"]=="San Francisco Bay Area"])
    #print(nfl_df[["team", "area"]].loc[nfl_df["team"].str.contains("49ers")])
    #print(mlb_df[["team", "area"]][mlb_df["area"]=="New York City"])
    #print(mlb_df[["team", "area"]].loc[mlb_df["team"].str.contains("Sox")])


    def group_df(x):
        df_merge = pd.merge(Big4[x][Big4[x]["year"]==2018][["area", "team", "W", "L", "year", "League"]],
                        cities["Metropolitan area"],
                        left_on="area", right_on="Metropolitan area")
        df_merge["W/L Ratio"] = df_merge["W"].astype(int) / (df_merge["W"].astype(int) + df_merge["L"].astype(int))
        grouped = df_merge.groupby("area").agg(Ratio=("W/L Ratio", np.mean))
        return grouped
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    ## // OBS al hacer stats.ttest_rel con grupos iguales resultado es NaN
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for x in Big4:
        for y in Big4:
            merge_df = pd.merge(group_df(x), group_df(y), how="inner", on="area")
            #print(merge.head(3))
            #print("For {} & {}".format(x, y))
            t_test, pval = stats.ttest_rel(merge_df["Ratio_x"], merge_df["Ratio_y"])
            p_values.loc[x, y] = pval

    #raise NotImplementedError()
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values


sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
